In [ ]:

%matplotlib inline
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyvista as pv
import vtk
import math

In [ ]:
wellLoc = pd.read_csv('VNC_BH_Loc.csv',index_col=0)
wellLoc.head()

In [ ]:
wellLito = pd.read_csv('VNC_BH_Lith.csv',index_col=0)
wellLito.head()

In [ ]:
litoPoints = []

for index, values in wellLito.iterrows():
    wellX, wellY, wellZ = wellLoc.loc[values.Borehole][["X","Y","Z"]]
    wellXY = [wellX, wellY]
    litoPoints.append(wellXY + [values.Z_Top,values.hydrogeoCode])
    litoPoints.append(wellXY + [values.Z_Bot,values.hydrogeoCode])
    
    litoLength = values.Z_Top - values.Z_Bot
    if litoLength < 1:
        midPoint = wellXY + [values.Z_Top - litoLength/2,values.hydrogeoCode]
    else:
        npoints = int(litoLength)
        for point in range(1,npoints+1):
            disPoint = wellXY + [values.Z_Top - litoLength*point/(npoints+1),values.hydrogeoCode]
            litoPoints.append(disPoint)
litoNp=np.array(litoPoints)
np.save("litoNp",litoNp)
litoNp[:5]

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing

In [ ]:
litoX, litoY, litoZ = litoNp[:,0], litoNp[:,1], litoNp[:,2]
litoMean = litoNp[:,:3].mean(axis=0)
litoTrans = litoNp[:,:3]-litoMean
litoTrans[:5]

#setting up scaler
scaler = preprocessing.StandardScaler().fit(litoTrans)
litoScale = scaler.transform(litoTrans)

#check scaler
print(litoScale.mean(axis=0))
print(litoScale.std(axis=0))

In [70]:
#run classifier
X = litoScale
Y = litoNp[:,3]
clf = MLPClassifier(activation='tanh',solver='lbfgs',hidden_layer_sizes=(30,30,30), max_iter=10000)
clf.fit(X,Y)

KeyboardInterrupt: 

In [ ]:
numberSamples = litoNp.shape[0]
expected=litoNp[:,3]
predicted = []
for i in range(numberSamples):
    predicted.append(clf.predict([litoScale[i]]))
results = confusion_matrix(expected,predicted)
print(results)

In [ ]:
xmin = 487028
xmax = 491162
ymin = 5144168
ymax = 5149592
zmax = int(wellLito.Z_Top.max())
zmin = zmax - 200

In [ ]:
#If index error occurs while constructing lithologic matrix (line 180), try reducing these values to increase grid resolution.
#As mentioned above with regards to rounding, higher number of non-zero digits in coordinate values appears to coincide with greater required discretization to avoid index error.

cellH = 20
cellV = 10

In [ ]:
vertexCols = np.arange(xmin,xmax+1,cellH)
vertexRows = np.arange(ymax,ymin-1,-cellH)
vertexLays = np.arange(zmax,zmin-1,-cellV)
cellCols = (vertexCols[1:]+vertexCols[:-1])/2
cellRows = (vertexRows[1:]+vertexRows[:-1])/2 
cellLays = (vertexLays[1:]+vertexLays[:-1])/2
nCols = cellCols.shape[0]
nRows = cellCols.shape[0]
nLays = cellLays.shape[0]

In [ ]:
i=0
litoMatrix=np.zeros([nLays,nRows,nCols])
for lay in range(nLays):
    for row in range(nRows):
        for col in range(nCols):
            cellXYZ = [cellCols[col],cellRows[row],cellLays[lay]]
            cellTrans = cellXYZ - litoMean
            cellNorm = scaler.transform([cellTrans])
            
            litoMatrix[lay,row,col] = clf.predict(cellNorm)
            
            if i%30000==0:
                print("Processing %s cells"%i)
                print(cellTrans)
                print(cellNorm)
                print(litoMatrix[lay,row,col])
            i+=1

In [ ]:
plt.imshow(litoMatrix[0])

In [ ]:
plt.imshow(litoMatrix[:,20])

In [ ]:
np.save('litoMatrix',litoMatrix)

In [ ]:
#matrix modification for Vtk representation
litoMatrixMod = litoMatrix[:,:,::-1]
np.save('litoMatrixMod',litoMatrixMod)
plt.imshow(litoMatrixMod[0])

In [ ]:
# Create empty grid
grid = pv.RectilinearGrid()

In [ ]:
# Initialize from a vtk.vtkRectilinearGrid object
vtkgrid = vtk.vtkRectilinearGrid()
grid = pv.RectilinearGrid(vtkgrid)
grid = pv.RectilinearGrid(vertexCols,vertexRows,vertexLays)

In [ ]:
litoFlat = list(litoMatrixMod.flatten(order="K"))[::-1]
grid.cell_arrays["hydrogeoCode"] = np.array(litoFlat)
grid.save('hydrogeologicalUnit.vtk')